# 파인튜닝 모델 추론 및 테스트

학습된 모델을 사용하여 용어 정의를 생성합니다.

---

## 1. 환경 설정

In [1]:
%%capture
!pip install transformers peft accelerate

In [2]:
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

사용 디바이스: cuda


In [3]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
MODEL_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/model/')

Mounted at /content/drive


## 2. 모델 로드

두 가지 방법 중 선택:
- **방법 1**: LoRA 어댑터만 로드 (메모리 효율적)
- **방법 2**: 병합된 전체 모델 로드 (더 간단)

In [4]:
# 방법 1: LoRA 어댑터 로드
BASE_MODEL = "skt/kogpt2-base-v2"
ADAPTER_PATH = MODEL_PATH / 'lora_adapter'

# 베이스 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map='auto'
)

# LoRA 어댑터 적용
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model = model.merge_and_unload()  # 추론 속도 향상을 위해 병합

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)

print("모델 로드 완료!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

모델 로드 완료!


In [5]:
# 방법 2: 병합된 모델 로드 (위의 방법 1 대신 사용 가능)
# MERGED_MODEL_PATH = MODEL_PATH / 'merged_model'
#
# model = AutoModelForCausalLM.from_pretrained(
#     MERGED_MODEL_PATH,
#     torch_dtype=torch.float16,
#     device_map='auto'
# )
# tokenizer = AutoTokenizer.from_pretrained(MERGED_MODEL_PATH)
#
# print("병합 모델 로드 완료!")

## 3. 추론 함수 정의

In [6]:
def generate_definition(term, domain="게임", facet=None, max_length=256):
    """
    용어의 정의를 생성합니다.

    Args:
        term: 정의할 용어
        domain: 분야 (게임, 레저, 미디어 등)
        facet: 세부 카테고리 (선택사항)
        max_length: 최대 생성 길이

    Returns:
        생성된 정의 문자열
    """
    # 프롬프트 생성
    if facet:
        instruction = f"다음은 {domain} 분야의 {facet} 관련 용어입니다. '{term}'의 정의를 설명해주세요."
    else:
        instruction = f"다음은 {domain} 분야의 용어입니다. '{term}'의 정의를 설명해주세요."

    prompt = f"""### 질문:
{instruction}

### 답변:
"""

    # 토큰화
    inputs = tokenizer(prompt, return_tensors='pt').to(device)

    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 답변 부분만 추출
    if "### 답변:" in generated_text:
        answer = generated_text.split("### 답변:")[1].strip()
    else:
        answer = generated_text[len(prompt):].strip()

    return answer

print("추론 함수 정의 완료!")

추론 함수 정의 완료!


## 4. 테스트

In [7]:
# 테스트할 용어 목록
test_terms = [
    {"term": "보스 몬스터", "domain": "게임", "facet": "몬스터"},
    {"term": "파밍", "domain": "게임", "facet": None},
    {"term": "던전", "domain": "게임", "facet": "지역"},
    {"term": "PvP", "domain": "게임", "facet": None},
    {"term": "버프", "domain": "게임", "facet": "스킬"},
]

print("=" * 60)
print("용어 정의 생성 테스트")
print("=" * 60)

for item in test_terms:
    print(f"\n용어: {item['term']}")
    print(f"   분야: {item['domain']}" + (f" / {item['facet']}" if item['facet'] else ""))
    print("-" * 40)

    definition = generate_definition(
        term=item['term'],
        domain=item['domain'],
        facet=item['facet']
    )

    print(f"   정의: {definition}")
    print()

용어 정의 생성 테스트

용어: 보스 몬스터
   분야: 게임 / 몬스터
----------------------------------------
   정의: 몬스터 헌터 월드 게임에서, 보스 몬스터인 보스의 이름.


용어: 파밍
   분야: 게임
----------------------------------------
   정의: 월드오브워크래프트 게임에서, 파밍을 통해 얻는 콘텐츠.


용어: 던전
   분야: 게임 / 지역
----------------------------------------
   정의: 디아블로2 레저렉션 게임에서, 던전의 이름.


용어: PvP
   분야: 게임
----------------------------------------
   정의: 월드오브워크래프트 게임에서, 특정 유닛의 체력을 회복시켜 주는 PVE.


용어: 버프
   분야: 게임 / 스킬
----------------------------------------
   정의: 리니지M 게임에서, 공격력 +4를 주는 방어구 아이템.



## 5. 인터랙티브 테스트

In [8]:
# 직접 용어를 입력해서 테스트해보세요!
term = "탱커"  # 원하는 용어로 변경
domain = "게임"  # 게임, 레저, 미디어 중 선택
facet = "직업"  # 선택사항 (None 가능)

print(f"\n'{term}' 정의 생성 중...\n")
result = generate_definition(term, domain, facet)
print(f"정의: {result}")


'탱커' 정의 생성 중...

정의: 몬스터 헌터 월드 게임에서, 탱커를 이르는 말.


## 6. 성능 평가 (선택사항)

In [10]:
import json
from pathlib import Path

# 검증 데이터로 평가
DATA_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/processed/')

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

val_data = load_jsonl(DATA_PATH / 'val.jsonl')
print(f"검증 데이터 수: {len(val_data)}")

검증 데이터 수: 5000


In [11]:
# 샘플 비교 (정답 vs 생성)
import random

sample_indices = random.sample(range(len(val_data)), 5)

print("=" * 70)
print("정답 vs 생성 결과 비교")
print("=" * 70)

for idx in sample_indices:
    item = val_data[idx]

    # 생성
    generated = generate_definition(
        term=item['term'],
        domain=item['domain'],
        facet=item.get('facet')
    )

    print(f"\n용어: {item['term']}")
    print(f"\n[정답]")
    print(f"{item['output'][:200]}..." if len(item['output']) > 200 else item['output'])
    print(f"\n[생성]")
    print(f"{generated[:200]}..." if len(generated) > 200 else generated)
    print("-" * 70)

정답 vs 생성 결과 비교

용어: 반동 폭탄

[정답]
리그오브레전드 게임에서, 직스가 튕기는 폭탄을 던져 적에게 피해를 입히는 기술.

[생성]
월드오브워크래프트 게임에서, '폭발 후 폭발하는 적에게 반격 피해를 주는 무기'를 줄여 이르는 말.
----------------------------------------------------------------------

용어: 얼큰이

[정답]
얼굴이 큰 사람을 놀림조로 이르는 말.

[생성]
얼큰하게 부풀어 오른 몸.
----------------------------------------------------------------------

용어: 첼로 협주곡

[정답]
첼로를 독주 악기로 하는 협주곡.

[생성]
첼로를 위한 곡.
----------------------------------------------------------------------

용어: 전통의상

[정답]
전통적인 방식대로 만든 의상.

[생성]
전통적인 옷을 입는 옷.
----------------------------------------------------------------------

용어: 남성 전용

[정답]
오로지 남성만 이용하거나 사용함.

[생성]
여성들이 남성을 대하는 태도나 행동.
----------------------------------------------------------------------


---
### 최종 결과물
```
Google Drive/LLM_Project/
├── data/                    # 원본 데이터
├── processed/               # 전처리된 데이터
│   ├── train.jsonl
│   └── val.jsonl
└── model/                   # 학습된 모델
    ├── lora_adapter/        # LoRA 어댑터
    ├── merged_model/        # 병합 모델
    └── checkpoints/         # 학습 체크포인트
```

### 보고서 작성 시 포함할 내용
1. **데이터셋**: AI Hub 문화/게임 콘텐츠 용어 말뭉치
2. **모델**: KoGPT2 + LoRA 파인튜닝
3. **학습 설정**: 에포크, 배치 사이즈, 학습률 등
4. **결과**: 생성 예시, 정성적 평가